# 09/30/2024: Calculate PCC: from google sheets

### 01 Access Google Sheets & Tabs *italicized text*

In [ ]:
# !pip install gspread google-auth google-auth-oauthlib google-auth-httplib2

import gspread
from google.colab import auth
from google.auth import default

# Authenticate and create the client
auth.authenticate_user()
creds, _ = default()
client = gspread.authorize(creds)

# Open the spreadsheet by URL
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/12DZi8qqYuKDJl90pZ6SJ2ZMzwE-Cd8Bx7LXQ9RXMnVw')

# Access and print the tabs (worksheets)
worksheet_list = sheet.worksheets()
for worksheet in worksheet_list:
    print(worksheet.title)


# 02 Read Columns and Values: CW, CX columns
Read specific columns from a Google Sheets tab using the gspread library. Here's a script to read the data from columns "CW" and "CX" of the tab named '10.4_DNA95':

In [ ]:
# Open the specific worksheet by name
worksheet = sheet.worksheet('10.4_DNA95')

# Fetch all values from columns CW and CX
cw_column = worksheet.col_values(101)  # CW is the 101st column
cx_column = worksheet.col_values(102)  # CX is the 102nd column

# Print the values from CW and CX
print("CW Column Values:")
for value in cw_column:
    print(value)

print("\nCX Column Values:")
for value in cx_column:
    print(value)


# 03 PCC calculation of CW-CX
To calculate the Pearson Correlation Coefficient (PCC) between the values in columns "CW" and "CX", you can use Python's scipy.stats library. Here's how you can modify the code to compute the PCC:

In [ ]:
# Import the necessary libraries
import numpy as np
from scipy.stats import pearsonr

# Open the specific worksheet by name
worksheet = sheet.worksheet('10.4_DNA95')

# Fetch all values from columns CW and CX
cw_column = worksheet.col_values(101)  # CW is the 101st column
cx_column = worksheet.col_values(102)  # CX is the 102nd column

# Convert the columns to numeric, ignoring any non-numeric values
cw_values = []
cx_values = []

for cw, cx in zip(cw_column, cx_column):
    try:
        cw_val = float(cw)
        cx_val = float(cx)
        cw_values.append(cw_val)
        cx_values.append(cx_val)
    except ValueError:
        # Ignore rows where values cannot be converted to float
        continue

# Calculate the Pearson Correlation Coefficient
if len(cw_values) > 1 and len(cx_values) > 1:
    pcc, _ = pearsonr(cw_values, cx_values)
    print(f"Pearson Correlation Coefficient (PCC) between CW and CX: {pcc}")
else:
    print("Not enough valid data points to calculate PCC.")


# 04 PCC calculation [final]

In [ ]:
# Import necessary libraries
import numpy as np
from scipy.stats import pearsonr

def get_column_index_from_label(label):
    """
    Convert a column label (e.g., 'A', 'B', ..., 'CW', 'CX') to a zero-based index.
    """
    col_index = 0
    for i, char in enumerate(reversed(label)):
        col_index += (ord(char) - ord('A') + 1) * (26 ** i)
    return col_index

def calculate_pcc(sheet_url, tab_name, col_label_1, col_label_2):
    """
    Calculate the Pearson Correlation Coefficient (PCC) between two columns in a Google Sheet tab.
    
    Parameters:
    - sheet_url: Google Sheets URL.
    - tab_name: The name of the tab (worksheet) to access.
    - col_label_1: The label (e.g., 'CW') of the first column.
    - col_label_2: The label (e.g., 'CX') of the second column.
    """
    
    # Authenticate and create the client (you can skip this if already authenticated)
    auth.authenticate_user()
    creds, _ = default()
    client = gspread.authorize(creds)

    # Open the spreadsheet by URL
    sheet = client.open_by_url(sheet_url)

    # Open the specific worksheet by name
    worksheet = sheet.worksheet(tab_name)

    # Convert column labels to 1-based indices
    col_index_1 = get_column_index_from_label(col_label_1)
    col_index_2 = get_column_index_from_label(col_label_2)

    # Fetch all values from the specified columns
    col_values_1 = worksheet.col_values(col_index_1)
    col_values_2 = worksheet.col_values(col_index_2)

    # Convert the columns to numeric, ignoring any non-numeric values
    values_1 = []
    values_2 = []

    for val_1, val_2 in zip(col_values_1, col_values_2):
        try:
            val_1_float = float(val_1)
            val_2_float = float(val_2)
            values_1.append(val_1_float)
            values_2.append(val_2_float)
        except ValueError:
            # Ignore rows where values cannot be converted to float
            continue

    # Check if there are enough valid data points to calculate PCC
    if len(values_1) > 1 and len(values_2) > 1:
        pcc, _ = pearsonr(values_1, values_2)
        print(f"Pearson Correlation Coefficient (PCC) between {col_label_1} and {col_label_2} in '{tab_name}' tab: {pcc}")
    else:
        print(f"Not enough valid data points to calculate PCC for columns {col_label_1} and {col_label_2}.")

# Example usage:
sheet_url = 'https://docs.google.com/spreadsheets/d/12DZi8qqYuKDJl90pZ6SJ2ZMzwE-Cd8Bx7LXQ9RXMnVw'
tab_name = '10.4_DNA95'  # The tab you want to work with
col_label_1 = 'CW'  # First column label
col_label_2 = 'CX'  # Second column label

calculate_pcc(sheet_url, tab_name, col_label_1, col_label_2)
